In [100]:
import pandas as pd
import numpy as np

In [101]:
df = pd.read_excel('elisha-data.xlsx')

In [102]:
columns = df.columns

In [103]:
filter_out = ['Unnamed: 0', 1, 2, 3, 4, 5, 6]

In [104]:
target_col = []
for col in columns:
    if col not in filter_out:
        target_col.append(col)
        

# Read in stuff from e-vape

In [376]:
import requests
import re
from bs4 import BeautifulSoup
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

In [106]:
URL = "https://www.elementvape.com/"
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

In [107]:
driver = webdriver.Firefox()
driver.get(URL)

In [108]:
legal_age = driver.find_element(By.CLASS_NAME, 'confirm-button')
legal_age.click()

### General Helper Functions

In [109]:
def load_everything(driver):
    load_more = driver.find_element(By.CLASS_NAME, 'amscroll-load-button')
    while True:
        try:
            load_more = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'amscroll-load-button')))
            time.sleep(2)
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
        except Exception as e:
            #print(e)
            print("no more to load")
            break

### Getting everything form disposables

In [255]:
disposable = driver.find_element(By.PARTIAL_LINK_TEXT, 'DISPOSABLES')
disposable.click()

In [256]:
load_everything(driver)

no more to load


In [257]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [258]:
#class="product-item-link"
disposable_dict = {}
products = soup.findAll('a', attrs = {'class':'product-item-link'}) 
for prod in products:
    disposable_dict[prod.text.strip()] = prod["href"]

In [270]:
with open("disposables.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerows(disposable_dict.items())

### Getting everything from Eqliquid

In [13]:
eliquids = driver.find_element(By.PARTIAL_LINK_TEXT, 'E-LIQUIDS')
eliquids.click()

In [14]:
flavor_profile = driver.find_elements(By.CLASS_NAME, 'filter-options-content')[9]
#flavors = flavor_profile.find_elements(By.TAG_NAME, 'input')
flavors = flavor_profile.find_elements(By.TAG_NAME, 'label')

In [16]:
flavor_list = []
for flavor in flavors:
    temp = flavor.text + ""
    temp = temp.lower().replace(" & ", "-")
    flavor_list.append(temp)

In [17]:
flavor_list

['fruit',
 'menthol-mint',
 'bakery',
 'custard-cream',
 'candy',
 'dessert',
 'nutty',
 'beverage',
 'breakfast',
 'tobacco']

In [31]:
wip = ["nutty", 'beverage', "breakfast", "tobacco"]

In [32]:
for flavor in flavor_list:
    print(flavor)
    eliquids_dict = {}
    
    #if flavor in wip: 
    url = "https://www.elementvape.com/e-liquids?flavor_profile=" + flavor
    driver.get(url)
    
    try:
        load_more = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'amscroll-load-button')))
        load_everything(driver)
    except:
        print("no more to load")
    
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    products = soup.findAll('a', attrs = {'class':'product-item-link'}) 
    for prod in products:
            eliquids_dict[prod.text.strip()] = prod["href"]

    with open("eliquids" + flavor + ".csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerows(eliquids_dict.items())

fruit
no more to load
menthol-mint
no more to load
bakery
no more to load
custard-cream
no more to load
candy
no more to load
dessert
no more to load
nutty
no more to load
beverage
no more to load
breakfast
no more to load
tobacco
no more to load


# Getting Brands

In [30]:
driver.get("https://www.elementvape.com/brands")

In [31]:
uppercase_letters = [chr(x) for x in range(ord('A'), ord('Z')+1)]

In [32]:
tag = "brands-letter letter-A"
tag = "letter-A"

temp = driver.find_element(By.CLASS_NAME, "brand-list")

In [34]:
temp = temp.text.split("\n")

In [36]:
temp = list(map(str.strip, temp))

In [79]:
brands = []
temp.append("JNR")
brand_regex = ""
for b in temp:
    if b not in uppercase_letters:
        brands.append(b)
        brand_regex = brand_regex + b + "|"

brand_regex = brand_regex[:len(brand_regex)-1]

In [80]:
with open("flavors.txt", "w", newline="") as f:
    for line in flavor_list:
        f.write(f"{line}\n")

In [81]:
with open("brands.txt", "w", newline="", encoding='utf-8') as f:
    for line in brands:
        f.write(f"{line}\n")

# Getting Descriptions

In [359]:
df2 = pd.read_csv('./disposables.csv', names=["name", "url"])  

#### Disposables Parsing

In [360]:
def helper_extract(search, description):
    interested_regex = re.compile(search)
    interested = interested_regex.search(description)
    if interested is not None:
        interested = interested.group(0).split(":")[1].strip()
    else:
        interested = ""
    return interested

In [384]:
def get_disposable_info(url):
    results = []
    try:
        driver.get(url)
    except:
        driver.get("https:" + url)
        
    time.sleep(2)
        
    title = driver.find_element(By.CLASS_NAME, "base")
    description_element = driver.find_element(By.ID, "description")
    results.append(title.text)

    brand_regex2 = re.compile(r"#|Fasta Burrst|7 Daze|AVCT Avictor|Advken|Air Factory|AirFuze|AirisTech|Airmez|Aleader|Aleaf|Aloha Sun|Arizer|Armageddon|Aspire|Astral Project|Astro Eight|Asvape|Augvape|Auxo|asMODus|BARZ|BLVK Unicorn|BP Mods|Bad Drip Labs|Bam Bam's Cannoli|Bantam Vape|Bar Juice|Beach Club Vapor|Beard Vape Co.|Binoid|Blank Bar|Blitz Enterprises|Bluegrass Cannabis Co.|Boulder Vape|Boundless|Bugatti|CBDfx|CCELL|CRAZYACE|Cake Vapors|Cali Alternatives|Cali Extrax|Candy King|Carats|Cartisan|Cereal Bar|Chapo Extrax|Charlie's Chalk Dust|Cheech|Chillax|Chromium Crusher|Chubby Gorilla|Cloud Nurdz|Cloudious9|Coastal Clouds|Coil Master|Craving Vapor|Crooked Creations|Cutleaf|Cuttwood|Czar|DAZZVAPE|DOGG LBS|DOVPO Technology|DOZO|DYNAVAP|DaVinci Vaporizers|Damn Vape|Daywalker|Dazzleaf|Death Row|Delta Extrax|Delta King|Delta Muchies|Diamond Supply Co.|Digiflavor|Dimo Hemp|Dotmod|Dr. Dabber|Duffle Bag Boyz|Dummy Vapes|dripMORE|ELF THC|ELUX Tech|ELYXR|EYCE|Echo|Efest|Eighty Six|Eleaf|Elysian Labs|Empire Glassworks|Equator|Exodus|FADED VAPES|FEELA|FLO|FRYD|FUME Vape|FUYL|Fifty Bar|Fire Vape|Firerose|Flayvorz|Flum Float|Flying Monkey|Focus V|Foger|Formula 420 Cleaners|Four Seasons|Freemax|Fresh Farms E-Liquid|Frozen Fields|Frutia|Fume Extracts|GRAV Labs|GRDNT|Galaxy Treats|Geek Vape|Geek'd Extracts|Ghost Hemp|Glamee|Glas Vapor|Glob Mops|Gold Bar Vapor|Goo'd Extracts|Good Girls Only|Good Vibez|Grenco Science|GÖST Vapor|HQD Technology|Half Bak'd|Hamilton Devices|Hangsen|Happi|Haze|Hellvape|Hemper|Hi Fog|Hi-Drip|HiXotic|Hidden Hills|Hideseek|High Times|Hohm Tech|Honeyroot Wellness|Honeystick|Horizon Tech|Hugo Vapor|Hulk Hogan|Humble Juice Co.|Huni Badger|INDACLOUD|ISPURE|IYKYK|Imperial Extraction|Innevape|Innokin|InstaBar|iJoy|iMini|JUS Vape|JUSTFOG|JUUL|Jeeter|Jimmy The JuiceMan|Joyetech|Juice Head|Juice King|Juice Roll Upz|Just CBD|KILO|KROS|Kado|Kangertech|Kangvape|Keep It 100|Koi CBD|Kush Burst|Kynn Labs|LG|LITTO|LUCID|LUFFBAR|LVE|Leaf Buddi|Lil Baby|Limitless Mod Co.|Logic|Lookah|Looper|Lost Mary|Lost THC|Lost Vape|Lucy|Lume|MECH VAPE|MJ Arsenal|MNKE BARS|MRKT|MTRX|Mad Hatter Juice|Mellow Fellow|Memory Lane|Merica Bar|Met4 Vapors|Metaku|Mighty Vapors|Mintopia|Mints Vape Co.|Mmelt|Modus|Monster Vape Labs|Mr. Freeze|NEXA|Naked 100 E-Liquid|Nasty Juice|Nitecore|Nitro's Cold Brew|Nomenon E-Liquids|North Vapes|Nose Knows|OBS|OEO|OLY|OMG|ON!|ONE Vape|ORGNX E-Liquids|OXBAR|OXVA|OZIBAR|Ocho Extracts|Off-Stamp|One Hit Wonder|Ooze|Oumier|PAX Labs|PLENA|Pachamama CBD|Phantoms|Piece Water|Pillow Talk|Pillowz|Pioneer4You|Ply Rock|Pod Juice|Pop Clouds|Posh Vape|Pressure LA|Prima|Propaganda E-Liquids|Puffco|Pulsar Vaporizers|Purlyf|Pushing P's|PynePod|Pyro|QIT|RAMA|RAW|RAZ|ROGUE|RUNTZ|RabBeats|Randy's|Releafy|Rick Flair Drip|Rincoe|Ruthless|SADBOY E-Liquid|SAVEURvape|SKWĒZED|SMOKtech|SOFI|STIIIZY|STNR|STORZ & BICKEL|STRIO|SVRF|SWFT|SWYPE|Salt Bae|Samsung|Santa Cruz|Sense|Shield|Shijin Vapor|Sigelei|SilverBack Juice Co|Smoant|Smoking Vapor|Snoopy Smoke|Snowwolf|Softglass|Solaris|Sony|South|Space Gods|Space Walker|Spaceman|Squid Industries|Steam Crave|Stratus|Stundenglass|Sugar Bar|Suicide Bunny|Suicide Mods|Suorin|Supercart|Sutra|Syn Liquids|TRE House|Terpboys|Tesla|The Finest E-Liquid|The Hemp Doctor|The Kind Pen|The Mamasan|Thunderhead Creations|Thursday|Tinted Brew Liquid Co.|Tiny Dancer|Tobeco|Torch|Treetop Hemp|Trinity Hemp|Trippy Sugar|True Story|Truweigh|Tsunami Premium|Twist E-Liquid|Tyson 2.0|UNO MAS|UPENDS|URB|UT BAR|Ultroner|Uwell|V Syndicate|VAPENGIN|VAPMOD|VEEPON|VELO|VGOD|VIHO|VNM|VOZOL|VPR|VUSE|Vandy Vape|Vape Breakfast Classics|Vape Dinner Lady|Vapefly|Vaperz Cloud|Vapetasia|Vaporesso|Vaptio|Velocity|Vessel|Viva La Hemp|Vladdin Vapor|VooPoo|Vyve|WAKA|Wake Mod Co.|White Rhino|Wismec|Wotofo|Wunder|XTRA|XVape|YOCAN|YOVO|Yatchvape|YiHi|Yo Bar|Yogi E-Liquids|Youde|ZIMO|ZYN|Zen Haus|Zombi|JNR", flags=re.IGNORECASE)

    brand = brand_regex2.search(title.text)
    if brand is not None:
        brand = brand.group(0)
    else:
        brand = "FIND BRAND"
    results.append(brand)

    product = title.text
    results.append(product)

    skunumber = ""
    results.append(skunumber)

    price_element = driver.find_element(By.CLASS_NAME, "price")
    price = price_element.text
    results.append(price)

    description = description_element.text
    results.append(description)

    eliquid_contents = helper_extract(r"PREFILLED CAPACITY(.*)(\r\n|\r|\n)", description)
    results.append(eliquid_contents)
    
    nicotine_level = helper_extract(r"NICOTINE STRENGTH(.*)(\r\n|\r|\n)", description)
    results.append(nicotine_level)
    
    puffs_per_device = helper_extract(r"MAX PUFFS(.*)(\r\n|\r|\n)", description)
    puffs_per_device_normal = puffs_per_device.split("|")
    if len(puffs_per_device_normal) > 1:
        puffs_per_device_normal = puffs_per_device_normal[1].strip()
    else: 
        puffs_per_device_normal = puffs_per_device_normal[0].strip()
    results.append(puffs_per_device)
    results.append(puffs_per_device_normal)
    
    battery = helper_extract(r"BATTERY CAPACITY(.*)(\r\n|\r|\n)", description)
    results.append(battery)
    
    try:
        flavor_options = driver.find_element(By.XPATH, "//*[contains(text(), 'AVAILABLE OPTIONS')]/parent::p")
        flavor_options = flavor_options.text.split("\n")
    
        if len(flavor_options) == 1:
            flavor_options = flavor_options[0].split(",")
        else:
            flavor_options.pop(0)

        flavor_options = [flavor.replace("• ", "") for flavor in flavor_options]
        flavor_options = ", ".join(flavor_options)
        results.append(flavor_options)
    except NoSuchElementException:
        try:
            flavor_options = driver.find_element(By.XPATH, "//*[contains(text(), 'Available Options')]/parent::p")
            flavor_options = flavor_options.text.split("\n")

            if len(flavor_options) == 1:
                flavor_options = flavor_options[0].split(",")
            else:
                flavor_options.pop(0)

            flavor_options = [flavor.replace("• ", "") for flavor in flavor_options]
            flavor_options = ", ".join(flavor_options)
            results.append(flavor_options)
        except:
            flavor_options = ""
            results.append(flavor_options)

    coil = helper_extract(r"HEATING ELEMENT(.*)(\r\n|\r|\n)", description)
    results.append(coil)

    led_regex = re.compile(r" screen| SCREEN")
    led = "yes" if led_regex.search(description) is not None else "no"
    results.append(led)

    dual_tank_disposable_vape = ""
    results.append(dual_tank_disposable_vape)

    smart_led = "no"
    if led == "yes":
        smart_led_regex = re.compile(r" touch screen| TOUCH SCREEN")
        smart_led = "yes" if smart_led_regex.search(description) is not None else "no"
    results.append(smart_led)

    turbo_mode = ""
    results.append(turbo_mode)
    
    disposable_vape_battery = battery
    results.append(disposable_vape_battery)

    charging_dock = helper_extract(r"CHARGING(.*)(\r\n|\r|\n)", description)
    results.append(charging_dock)
    
    full_led_screen = led
    results.append(full_led_screen)

    power_mode_setting = ""
    results.append(power_mode_setting)

    airflow = helper_extract(r"AIRFLOW(.*)(\r\n|\r|\n)", description)
    results.append(airflow)

    operation = helper_extract(r"OPERATION(.*)(\r\n|\r|\n)", description)
    results.append(operation)
    
    results.append(url)
    
    return results

#### The actual thing

In [362]:
columns = ["title", "brand", "product", "skunumber", "price", "description", "eliquid_contents", "nicotine_level", "puffs_per_device", "puffs_per_device_normal", "battery", "flavor_options", "coil", "led", "dual_tank", "smart_led", "turbo", "disposable_vape_battery", "charging_dock", "full_led_screen", "power_mode_setting", "airflow", "operation", "url"]

extracted_data = pd.DataFrame(columns=columns)

In [385]:
len(extracted_data)

102

In [380]:
df2

,name,url
102,Mystery Disposable Vape Pack,https://www.elementvape.com/mystery-disposable...
103,PLENA 18K Disposable,https://www.elementvape.com/plena-18k-disposable
104,iJoy TaijiZen Judo 24000 Disposable,https://www.elementvape.com/ijoy-taijizen-judo
105,Airis Alpha Touch 20000 Disposable,https://www.elementvape.com/airis-alpha-touch
106,VOZOL Gear Power 20000 Disposable,https://www.elementvape.com/vozol-gear-power
...,...,...
417,Blu Gold Leaf Disposable Pods,https://www.elementvape.com/blu-gold-leaf-disp...
418,ELF CBD5000 Disposable,https://www.elementvape.com/elf-cbd5000-dispos...
419,[10-Pack] SMOK Novo Bar AL6000 Disposable,https://www.elementvape.com/smok-novo-bar-10pk
420,BLANK BAR 15K Disposable Pod System,https://www.elementvape.com/blank-bar-15k-disp...


In [391]:
from tqdm import tqdm
   
for i in tqdm(range(len(df2))):
    current = df2.iloc[i]
    url = current["url"]
    temp = get_disposable_info(url)

    #disposables_list.append(temp)
    extracted_data.loc[len(extracted_data)] = temp
    df2 = df2.iloc[1: , :]

 50%|████████████████████████████████████████▌                                        | 80/160 [12:50<12:50,  9.63s/it]


IndexError: single positional indexer is out-of-bounds

In [392]:
extracted_data.to_csv('disposables_extracted.csv', index=False) 